<a href="https://colab.research.google.com/github/GeraudBourdin/llm-scripts/blob/main/Axolotl_QLoRA_Mistral7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Axolotl

# **Commandes utiles**

**Finetuning**
```
accelerate launch -m axolotl.cli.train mistral-orca-config.yml
```

**Inference** le model que l'on vient de finetuné
```
accelerate launch -m axolotl.cli.inference mistral-orca-config.yml --sample_packing False
```

**Debug** Pour afficher les promtps sur lesquelles le modèle est en cours Finetuning
```
accelerate launch -m axolotl.cli.train mistral-orca-config.yml --debug_text_only --debug
```

**Debug** avec les la vidu des tokens --prepare_ds_only
```
accelerate launch -m axolotl.cli.train mistral-orca-config.yml --prepare_ds_only --debug
```

**Fusionner QLoRA** avec son model de reference
```
python3 -m axolotl.cli.merge_lora mistral-orca-config.yml --lora_model_dir="./finetuned-model" --load_in_8bit=False --load_in_4bit=False
```



# Installation

In [ ]:
### sur jupyter

!git clone https://github.com/OpenAccess-AI-Collective/axolotl
%cd axolotl
# !pip install torch==2.1.2
# !pip install -r requirements.txt
!pip install packaging
!pip install -e '.[flash-attn,deepspeed]'
!pip install huggingface_hub

!git config --global credential.helper store
!wandb login --relogin
!huggingface-cli login

In [ ]:
##### sur colab
!pip install torch=="2.1.2"
!pip install -e git+https://github.com/OpenAccess-AI-Collective/axolotl#egg=axolotl
!pip install flash-attn=="2.5.0"
!pip install deepspeed=="0.13.1"
!pip install huggingface_hub

!git config --global credential.helper store
!wandb login --relogin
!huggingface-cli login

# Configuration

In [ ]:
contenu = """
base_model: mistralai/Mistral-7B-Instruct-v0.2
model_type: MistralForCausalLM
tokenizer_type: LlamaTokenizer

load_in_8bit: false
load_in_4bit: true
strict: false

datasets:
  - path: /content/train.json
    ds_type: json
    type: context_qa.load_v2
output_dir: ./qlora-out_customer

adapter: qlora
lora_model_dir:

sequence_len: 8192
sample_packing: true
pad_to_sequence_len: true

lora_r: 32
lora_alpha: 16
lora_dropout: 0.05
lora_target_linear: true
lora_fan_in_fan_out:
lora_target_modules:
  - gate_proj
  - down_proj
  - up_proj
  - q_proj
  - v_proj
  - k_proj
  - o_proj

wandb_project: partitech
wandb_entity:
wandb_watch:
wandb_name: partitech__customer_axolotl
wandb_log_model:

hf_use_auth_token: true
hub_strategy: checkpoint
hub_model_id: partitech/test_axolotl3


gradient_accumulation_steps: 4
micro_batch_size: 2
num_epochs: 4
optimizer: adamw_bnb_8bit
lr_scheduler: cosine
learning_rate: 0.0002

train_on_inputs: false
group_by_length: false
bf16: auto
fp16:
tf32: false

gradient_checkpointing: true
early_stopping_patience:
resume_from_checkpoint:
local_rank:
logging_steps: 1
xformers_attention:
flash_attention: true

loss_watchdog_threshold: 5.0
loss_watchdog_patience: 3

warmup_steps: 10
evals_per_epoch: 1
eval_table_size:
eval_max_new_tokens: 128
saves_per_epoch: 1
debug:
deepspeed:
weight_decay: 0.0
fsdp:
fsdp_config:
special_tokens:

resume_from_checkpoint: /content/qlora-out_customer/checkpoint-1083
"""
##### si besoin de resumer depuis le dernier checkpoint:
##### resume_from_checkpoint: ./qlora-out_customer/checkpoint-xxxx


with open("/content/partitech.yaml", "w") as fichier:
    fichier.write(contenu)

!cat /content/partitech.yaml
!head /content/train.json

In [ ]:
!accelerate launch -m axolotl.cli.train /content/partitech.yaml  --debug_text_only --debug

#Relancer un train pour continuer des epochs


Pour relancer un epoch sur les fichier soit on recupere les fichier dans le repertoire checkpointxxxxx

Soit si il sont sur huggingface on recupere les fichiers avec

In [ ]:
!pip install huggingface_hub



In [ ]:
from huggingface_hub import Repository
repo = Repository(local_dir="test_epoch4", clone_from="partitech/test_axolotl2")